In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC, LinearSVC

In [2]:
df = pd.read_csv('../data/weblogs.csv')

df = df.drop(['NIGHT', 'ID', 'OTHER_METHOD'], axis=1)

X = df.loc[:, df.columns != 'ROBOT']
y = df['ROBOT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

### Linear SVM Classification

In [3]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'loss': ['hinge', 'squared_hinge'],
    'C': [0.1, 1, 10, 100],
    'dual': [True, False],
    'fit_intercept': [True, False],
    'class_weight': [None, 'balanced']
}

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('grid_search', GridSearchCV(LinearSVC(random_state=42), param_grid, verbose=3, n_jobs=-1, cv=5))
])


In [4]:
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Fitting 5 folds for each of 128 candidates, totalling 640 fits


c:\Users\buyse\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
320 fits failed out of a total of 640.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\buyse\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\buyse\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py", line 274, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
                                           ^^^^^^^^^^^^^^^
  File

              precision    recall  f1-score   support

           0       0.98      0.95      0.97     10785
           1       0.82      0.94      0.87      2686

    accuracy                           0.95     13471
   macro avg       0.90      0.94      0.92     13471
weighted avg       0.95      0.95      0.95     13471

[[10219   566]
 [  161  2525]]


c:\Users\buyse\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [5]:
pipeline.named_steps['grid_search'].best_estimator_

LinearSVC(C=0.1, loss='hinge', random_state=42)

In [6]:
final_svc = pipeline.named_steps['grid_search'].best_estimator_

final_svc.fit(X_train, y_train)

y_pred = final_svc.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

print("Accuracy: {:.2f}".format(((accuracy_score(y_test, y_pred)) * 100)) + "%")

              precision    recall  f1-score   support

           0       0.86      0.99      0.92     10785
           1       0.89      0.33      0.48      2686

    accuracy                           0.86     13471
   macro avg       0.87      0.66      0.70     13471
weighted avg       0.86      0.86      0.83     13471

[[10672   113]
 [ 1798   888]]
Accuracy: 85.81%


c:\Users\buyse\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [7]:
import pickle

pickle.dump(final_svc, open("../modellen/svc_lin.sav", 'wb'))